In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [2]:
#!pip install adanet


In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [4]:
!pwd

/home/bangui/Dropbox/Perso/Formation/openclassrooms/OC_Datascientist/P8


In [5]:
datadir = './data'

In [6]:
datadir

'./data'

In [7]:
import p5_util
import p8_util

filename=datadir+'/arr_keras_X_y_train_test.dump'
(x_train,x_test, y_train, y_test) = p5_util.object_load(filename)
if True :
    y_train=p8_util.array_label_encode_from_index(y_train)
    y_test=p8_util.array_label_encode_from_index(y_test)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

w_size = x_train.shape[1]
h_size = x_train.shape[2]        

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414,) (47,)


In [8]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)
y_train.min(), y_train.max(), x_train.min(),  x_train.max()
y_train[0],y_test[0]

y_train[0], y_test[1]
l = y_train[0].shape
len(l)
a=(11,)
len(y_train.shape)
if 1 ==  len(y_train.shape) :
    nClasses=1
else :
    nClasses = y_train.shape[1]
print("Number of classes= "+str(nClasses))


Number of classes= 1


In [9]:
#help(adanet.subnetwork.Generator)
y_train.shape

(414,)

In [10]:
import tensorflow as tf
import p8_util
import shutil

    
LEARNING_RATE = 0.003  #@param {type:"number"}
TRAIN_STEPS = 4  #@param {type:"integer"}
BATCH_SIZE = 49  #@param {type:"integer"}

ADANET_ITERATIONS = 2  #@param {type:"integer"}
NN_TYPE = 'CNN'

tuple_dimension = (w_size, h_size,3)
nClasses=3

my_feature_columns, loss_reduction, tf_head \
= p8_util.get_tf_head("images",tuple_dimension, nClasses)

# A `Head` instance defines the loss function and metrics for `Estimators`.
# Tells Tensorfow how to compute loss function and metrics
#head = tf.contrib.estimator.multi_class_head(nClasses\
#                                             , loss_reduction=loss_reduction)
estimator = adanet.Estimator(
    head=tf_head,
    
    subnetwork_generator=p8_util.MyGenerator(
        nClasses,
        feature_columns=my_feature_columns,
        optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
        seed=p8_util.RANDOM_SEED,
        nn_type=NN_TYPE),
        
    
    max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , tuple_dimension=tuple_dimension\
                          , training=False\
                          , batch_size=BATCH_SIZE),
        steps=None),
    config=p8_util.make_config(NN_TYPE))

*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/models/CNN', '_tf_random_seed': 42, '_save_summary_steps': 100000, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa31a58a0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train", x_train, y_train\
                           , tuple_dimension=tuple_dimension\
                           , training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test", x_test, y_test\
                           , tuple_dimension=tuple_dimension\
                           , training=False, batch_size=BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    ))
print("Accuracy:", results["accuracy"])
print("Loss:", results["average_loss"])

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 100000 or save_checkpoints_secs None.
INFO:tensorflow:Beginning training AdaNet iteration 0
INFO:tensorflow:Calling model_fn.

*** generate_candidates : Iteration= 0 /Layers=0

*** _NNAdaNetBuilder : Classes=3

*** _NNAdaNetBuilder : Classes=3
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork 'CNN_layer_0'


*** build_subnetwork() : features shape= (?, 224, 224, 3)
*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3
Instructions for updating:
`persisted_tensors` is deprecated, please use `shared` instead.

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_fi

INFO:tensorflow:Building subnetwork 'CNN_layer_1'


*** build_subnetwork() : features shape= (?, 224, 224, 3)
*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3

*** *** Last layer shape= Tensor("adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/Flatten/flatten/Reshape:0", shape=(?, 401408), dtype=float32)

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3
INFO:tensorflow:Warm-starting from: ('./tmp/models/CNN/model.ckpt-2',)
INFO:tensorflow:Warm-starting variable: global_step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_C

INFO:tensorflow:Building subnetwork 'CNN_layer_1'


*** build_subnetwork() : features shape= (?, 224, 224, 3)
*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3

*** *** Last layer shape= Tensor("adanet/iteration_1/ensemble_t1_CNN_layer_1/weighted_subnetwork_1/subnetwork/Flatten/flatten/Reshape:0", shape=(?, 401408), dtype=float32)

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3
INFO:tensorflow:Building subnetwork 'CNN_layer_2'


*** build_subnetwork() : features shape= (?, 224, 224, 3)
*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3

*** *** Last layer shape= Tensor("adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/Flatten/flatten/Reshape:0", shape=(?, 401408), dtype=float32)

*** *** Last layer shape= Tensor("adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/Flatten_1

In [17]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS

filenale_model_check = './tmp/models/'+str(NN_TYPE)+'/model.ckpt-3'
#print_tensors_in_checkpoint_file(file_name=filenale_model_check, tensor_name='', all_tensors=False)
print("\n----------\n")
filenale_model_check = './tmp/models/'+str(NN_TYPE)+'/model.ckpt-'+str(max_iteration_steps)
print_tensors_in_checkpoint_file(file_name=filenale_model_check, tensor_name='', all_tensors=False)



----------

adanet/iteration_0/candidate_t0_CNN_layer_1/adanet/iteration_0/candidate_t0_CNN_layer_1/adanet_loss/biased (DT_FLOAT) []
adanet/iteration_0/candidate_t0_CNN_layer_1/adanet/iteration_0/candidate_t0_CNN_layer_1/adanet_loss/local_step (DT_FLOAT) []
adanet/iteration_0/candidate_t0_CNN_layer_1/adanet_loss (DT_FLOAT) []
adanet/iteration_0/ensemble_t0_CNN_layer_1/bias (DT_FLOAT) [3]
adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/logits/mixture_weight (DT_FLOAT) []
adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d/bias (DT_FLOAT) [64]
adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d/kernel (DT_FLOAT) [3,3,3,64]
adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d_1/bias (DT_FLOAT) [128]
adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d_1/kernel (DT_FLOAT) [3,3,64,128]
adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/sub

In [18]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [19]:
print("Architecture:", ensemble_architecture(results))

Architecture: b'| CNN_layer_1 | CNN_layer_2 |'


In [15]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNN_layer_1 |'  : accuracy = 32% / Loss =  1.0998582
# Architecture: b'| CNN_layer_0 |'  : accuracy = 45% / Loss =  1.0836046

In [16]:
results['architecture/adanet/ensembles']

b'\no\n>adanet/iteration_1/ensemble_t1_CNN_layer_2/architecture/adanetB#\x08\x07\x12\x00B\x1d| CNN_layer_1 | CNN_layer_2 |J\x08\n\x06\n\x04text'